In [1]:
import numpy as np
import pandas as pd
from apyori import apriori

In [2]:
path = '..'

target = "C:/Jupyter/aidea-farmland-crops-autumn/evaluate/big_transfer/BiT-M-R50x1_batch256_lr0.0003_fullset/pred_result_fullset.pkl"

In [4]:
data = pd.read_pickle(target)
data = data[data['set_name'].eq('test')].reset_index(drop=True)
data

,file,label,set_name,predict,predict_label
0,5175e60c-6cec-4768-b263-5dd2e1461318.jpg,lemon,test,"[8.1059276e-05, 0.00015124626, 0.010726868, 8....",lemon
1,989dfdc4-8220-4746-8537-5bd6b13e5333.jpg,lemon,test,"[0.00012853765, 5.3790485e-05, 7.280936e-05, 0...",lemon
2,7932eb30-4035-4ff0-8d37-da527f6c87a8.jpg,lemon,test,"[0.0012976688, 0.072816715, 0.12271332, 0.0007...",lemon
3,7bf8c3dc-1aa6-4846-b1b8-a73f7168053d.jpg,lemon,test,"[0.0001262426, 3.8281425e-05, 0.00017651086, 0...",lemon
4,e4337a76-8fbb-422f-a21a-d43f975aeec2.jpg,lemon,test,"[0.00040089124, 0.00015607402, 0.00024498475, ...",lemon
...,...,...,...,...,...
13449,d41c1fe0-1775-4940-835a-7e18ae079267.jpg,greenonion,test,"[7.358785e-05, 0.00015296991, 0.000106066844, ...",greenonion
13450,9e318bca-3186-4458-8ed9-b1fc53cc2952.jpg,greenonion,test,"[0.00018206096, 0.00028757274, 0.00016238686, ...",greenonion
13451,d2ff6ca0-7ad0-48a3-beb9-f168d527f277.jpg,greenonion,test,"[0.00045041938, 0.0018955074, 0.0033849792, 0....",greenonion
13452,74af1a5a-c258-49de-bfbe-1ccc659d5d80.jpg,greenonion,test,"[0.0006068007, 6.7271336e-05, 0.0002129048, 0....",greenonion


In [5]:
labels = data["label"].drop_duplicates().sort_values().reset_index(drop=True)

In [35]:
pred_error_df = data[data['label'].ne(data['predict_label'])].reset_index(drop=True)
pred_error_count_df = pred_error_df.groupby(['label', 'predict_label']).size().reset_index(name='counts')
pred_error_count_df['percentage'] = pred_error_count_df['counts'] / pred_error_count_df.groupby(['label'])['counts'].transform('sum')
pred_error_count_df[pred_error_count_df['percentage']> 0.3].sort_values(by=['counts'], ascending=False, ignore_index=True)

,label,predict_label,counts,percentage
0,cauliflower,broccoli,94,0.686131
1,broccoli,cauliflower,45,0.671642
2,greenonion,onion,40,0.500000
3,litchi,longan,37,0.528571
4,chinesecabbage,lettuce,26,0.464286
5,onion,greenonion,25,0.806452
6,kale,chinesecabbage,24,0.333333
7,longan,litchi,23,0.433962
8,soybeans,redbeans,20,0.444444
9,greenhouse,others,19,0.395833


In [43]:
association_rules = list(apriori(data[['label', 'predict_label']].values, min_support=0.003, min_lift = 1.1))
association_rules = pd.DataFrame(association_rules).sort_values(by=['support'], ascending=False, ignore_index=True)
association_rules

,items,support,ordered_statistics
0,"(broccoli, cauliflower)",0.010331,"[((broccoli), (cauliflower), 0.228995057660626..."
1,"(greenonion, onion)",0.004831,"[((greenonion), (onion), 0.11796733212341198, ..."
2,"(litchi, longan)",0.004460,"[((litchi), (longan), 0.1466992665036675, 4.39..."
3,"(kale, cauliflower)",0.003270,"[((cauliflower), (kale), 0.10185185185185185, ..."
4,"(chinesecabbage, lettuce)",0.003122,"[((chinesecabbage), (lettuce), 0.1105263157894..."


In [51]:
association_rules.to_csv(F"{path}/data/association_rules.csv", index=False)

## Data Split

In [3]:
# Top level data directory. Here we assume the format of the directory conforms to the ImageFolder structure
path = "."
# path = "/content/drive/MyDrive/Colab Notebooks/aidea-farmland-crops-autumn"

# Folder of the training data
training_folder = 'D:/Dataset/aidea-farmland-crops-autumn/train'

# Folder of the public data
public_folder = 'D:/Dataset/aidea-farmland-crops-autumn/public'

# Number of classes in the dataset
num_classes = 33

# Image size for model
image_size = 224

# options: (subset, fullset)
train_mode = 'fullset'

# options: (crop70xy, correct_by_rule, crop70xy_correct_by_rule)
image_preprocessing_method ="crop70xy_correct_by_rule"

# Get the shape of the input layer
input_shape = (image_size, image_size, 3)

In [4]:
association_rules = pd.read_csv(F"{path}/data/association_rules.csv")
association_rules['items'] = association_rules['items'].apply(lambda x: x.replace("frozenset({", "").replace("})", "").replace("'", "").split(", "))
association_rules

,items,support,ordered_statistics
0,"[broccoli, cauliflower]",0.010331,[OrderedStatistic(items_base=frozenset({'brocc...
1,"[greenonion, onion]",0.004831,[OrderedStatistic(items_base=frozenset({'green...
2,"[litchi, longan]",0.004460,[OrderedStatistic(items_base=frozenset({'litch...
3,"[kale, cauliflower]",0.003270,[OrderedStatistic(items_base=frozenset({'cauli...
4,"[chinesecabbage, lettuce]",0.003122,[OrderedStatistic(items_base=frozenset({'chine...


In [5]:
train = pd.read_pickle(F'{path}/data/{train_mode}_train_vector_{image_size}_{image_preprocessing_method}.pkl')
valid = pd.read_pickle(F'{path}/data/{train_mode}_valid_vector_{image_size}_{image_preprocessing_method}.pkl')
test = pd.read_pickle(F'{path}/data/{train_mode}_test_vector_{image_size}_{image_preprocessing_method}.pkl')
print(F"train: {train.shape}, valid: {valid.shape}, test: {test.shape}")

train: (62670, 22), valid: (13390, 22), test: (13454, 22)


In [6]:
for row in association_rules.itertuples():
    sub_train = train[train['label'].isin(row.items)].reset_index(drop=True)
    sub_valid = valid[valid['label'].isin(row.items)].reset_index(drop=True)
    sub_test = test[test['label'].isin(row.items)].reset_index(drop=True)
    sub_train.to_pickle(F'{path}/data/binary/{train_mode}_train_vector_{image_size}_{image_preprocessing_method}_{row.items[0]}_{row.items[1]}.pkl', protocol=4)
    sub_valid.to_pickle(F'{path}/data/binary/{train_mode}_valid_vector_{image_size}_{image_preprocessing_method}_{row.items[0]}_{row.items[1]}.pkl', protocol=4)
    sub_test.to_pickle(F'{path}/data/binary/{train_mode}_test_vector_{image_size}_{image_preprocessing_method}_{row.items[0]}_{row.items[1]}.pkl', protocol=4)